In [1]:
from torch.utils.tensorboard import SummaryWriter
import os
import surpport.mySQL as mySQL
import surpport.myfunction as MF
from surpport.Args import Args
from surpport.dataprocess import dataload
import torch.nn.functional as F
import torch
import numpy as np

Using backend: pytorch


## description
该文件主要是规整了模型多次训练的流程，函数化封装以完成接下来的各种任务

In [ ]:
import sys
from importlib import reload

In [ ]:
# sys.path.append(r'E:\CODEBASE\myDGL\FirstDGL\Experiment\Baseline\NoneGraph')
# import EEGNet
# reload(EEGNet)
# sys.path.append(r'E:\CODEBASE\myDGL\FirstDGL\Experiment\Baseline\GraphModel\GNN')
import GNN
reload(GNN)

In [ ]:
class Transformer(torch.nn.Module):
    def __init__(self) -> None:
        super(Transformer, self).__init__()
        t_layer = torch.nn.TransformerEncoderLayer(18, 3, 128,)
        self.transformer = torch.nn.TransformerEncoder(t_layer, 5)
        self.classifier = torch.nn.Linear(18, 5)
        self.norm = torch.nn.BatchNorm1d(18)

    def forward(self, graph):
        h1 = graph.ndata['h'].view(-1, graph.batch_size, 18)
        h1 = self.transformer(h1)
        h1 = self.norm(torch.sum(h1, dim=0).squeeze())
        return self.classifier(h1), 0

In [ ]:
dec_base = [0.00001, 0.00003]
w_dec = []
for i in range(13):
    w_dec.append(dec_base[i%2] * 10**(i//2))

w_dec

In [2]:
from surpport.nnstructure import Classifier7
def m_pre():
    arg = Args()
    # model = EEGNet.EEGNet2()
    model = Classifier7(arg)
    # model = GNN.DGCN()
    # model = Transformer()
    model = model.cuda()
    return arg, model

def recorder_build(arg):
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    return recorder

In [3]:
names = ['kly']
fs = [2, ]
for name in names:
    for f in fs:
        arg = Args()
        arg.pip_num = 3
        arg.dr = 0
        model = Classifier7(arg)
        model = model.cuda()
        arg.d_prepare(name, f)
        # ! 每次运行都改
        arg.m_info(m_name='m7', m_task='220909_wd_test', num=1,)
        base_rcd = mySQL.gen_base_rcd(arg)
        recorder = {'base': base_rcd}
        writer = SummaryWriter(arg.tar_path+'\\Journal')

        # ! 根据模型修改
        opt_arg = {'params': model.parameters(), 'weight_decay': 0.1}
        # opt_arg = {'params': model.parameters(),'lr': 6e-5, 'eps': 1e-8, 'weight_decay': 0.1}
        opt = torch.optim.Adam(**opt_arg)

        arg, tr_dataloader, ts_dataloader = dataload(arg, model, opt)

In [ ]:
from importlib import reload
import surpport.nnstructure as mynn

arg.pip_num = 1
arg.dr = 0
model = mynn.Classifier7(arg)
model = model.cuda()

In [7]:
st = 151
step = 50
for epoch in range(st, step+st):
    print(epoch)
    recorder[str(epoch)+'-th'] = dict()
    rd = recorder[str(epoch)+'-th']
    # 由于图经过拼合，所以需要多一个dataloader的过程
    # 前两个是list
    tr_args = [epoch, model, opt, tr_dataloader, arg, writer]
    tr_loss, tr_acc = MF.train(*tr_args)
    mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

    tr_args[3] = ts_dataloader
    el_loss, el_acc, logits, labels = MF.evaluate(*tr_args)

    mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
    mySQL.rcd_result(logits, labels, rd)

    val_acc = np.mean(el_acc, axis=0)
    MF.save_best(val_acc, model, arg)

    mySQL.save_final(epoch, model, val_acc, arg, opt)

mySQL.save_recorder(recorder, arg, 'flow')
print('best acc: %.4f' % (arg.best_acc))


151
Epoch 151, Iter 005,train loss = 0.7160, train acc = 0.6875
Epoch 151, Iter 010,train loss = 0.6212, train acc = 0.8125
Epoch 151, Iter 015,train loss = 0.6150, train acc = 0.7500
Epoch 151, Iter 020,train loss = 0.6758, train acc = 0.8438
Epoch 151, Iter 025,train loss = 0.6895, train acc = 0.7812
Epoch 151, Iter 030,train loss = 0.6225, train acc = 0.8750
Epoch 151, Iter 035,train loss = 0.6363, train acc = 0.7812
Epoch 151, Iter 040,train loss = 0.5705, train acc = 0.7500
Epoch 151, Iter 045,train loss = 0.5378, train acc = 0.8438
Epoch 151, val loss = 0.6401, val acc = 0.7673
未达到期望，未保存模型
152
Epoch 152, Iter 005,train loss = 0.6099, train acc = 0.6562
Epoch 152, Iter 010,train loss = 0.5485, train acc = 0.7812
Epoch 152, Iter 015,train loss = 0.5015, train acc = 0.8750
Epoch 152, Iter 020,train loss = 0.4919, train acc = 0.9062
Epoch 152, Iter 025,train loss = 0.6170, train acc = 0.6562
Epoch 152, Iter 030,train loss = 0.7167, train acc = 0.7812
Epoch 152, Iter 035,train loss = 

In [ ]:
writer.close()